In [1]:
import sys

In [2]:
sys.path.append("MSongsDB/PythonSrc/")

In [3]:
import hdf5_getters as hd

In [4]:
import os
import sqlite3
import numpy as np
import io



def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)


# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)


conn = sqlite3.connect("music.db", detect_types=sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS music (pk INTEGER PRIMARY KEY ASC, artist TEXT, title TEXT, duration FLOAT, segments_start array, segments_pitches array, section_start array)")


for dp, dn, fn in os.walk("/Users/sriram/Downloads/MillionSongSubset"):
    for f in fn:
        if f.endswith("h5"):
            curr = hd.open_h5_file_read(os.path.join(dp, f))
            cur.execute("INSERT INTO music(artist, title, duration, segments_start, segments_pitches, section_start) VALUES (?, ?, ?, ?, ?, ?)", (hd.get_artist_name(curr).decode('utf-8'), 
                        hd.get_title(curr).decode('utf-8'), hd.get_duration(curr), hd.get_segments_start(curr), hd.get_segments_pitches(curr), 
                        hd.get_sections_start(curr)))

In [5]:
import pandas as pd
import sqlite3

df = pd.read_sql_query("SELECT * from music", conn)

df.head()



,pk,artist,title,duration,segments_start,segments_pitches,section_start
0,1,Raphaël,Je Sais Que La Terre Est Plate,148.74077,"[0.0, 0.19188, 0.60499, 0.84939, 1.10018, 1.36...","[[0.726, 0.112, 0.02, 0.019, 0.01, 0.038, 0.03...","[0.0, 7.74174, 40.30707, 80.4797, 89.656, 127...."
1,2,Julie Zenatti,On Efface,252.99546,"[0.0, 0.51379, 0.89138, 1.30912, 1.65297, 2.05...","[[0.943, 1.0, 0.91, 0.782, 0.726, 0.806, 0.314...","[0.0, 7.08047, 39.03442, 89.26989, 114.94898, ..."
2,3,The Baltimore Consort,Howells Delight,78.02730,"[0.0, 0.97365, 1.34431, 1.81351, 2.19764, 2.44...","[[1.0, 0.926, 0.264, 0.384, 0.298, 0.261, 0.22...","[0.0, 13.98322, 29.36585, 42.60077]"
3,4,I Hate Sally,Martha Served,163.63057,"[0.0, 0.07306, 0.36862, 0.64739, 0.79238, 1.04...","[[0.835, 0.639, 0.492, 0.399, 0.326, 0.697, 0....","[0.0, 8.33166, 37.94071, 63.53617, 82.18998, 9..."
4,5,Orlando Pops Orchestra,Zip-A-Dee-Doo-Dah (Song of the South),199.99302,"[0.0, 0.3166, 0.82195, 1.2751, 1.80871, 2.3314...","[[0.014, 0.014, 0.246, 0.037, 0.078, 0.322, 0....","[0.0, 8.42605, 32.81331, 64.19633, 74.73863, 1..."


In [6]:
l = df.sort_values(by=['segments_pitches'], ascending=True, key=lambda x: x.map(lambda y: y.shape[0]))
l2 = df.sort_values(by=['duration'], ascending=True)

ls = [r[5].shape[0] for i,r in l[2000:4000].iterrows() ]


newdf = l[2000:4000]

newdf.head(50)

/var/folders/_n/5wvq9z1s4332vpfqd195sdy80000gn/T/ipykernel_11062/349192233.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ls = [r[5].shape[0] for i,r in l[2000:4000].iterrows() ]


,pk,artist,title,duration,segments_start,segments_pitches,section_start
4850,4851,Daniel Johnston,High Horse,153.33832,"[0.0, 0.19438, 0.51392, 0.66993, 1.14014, 1.44...","[[1.0, 0.156, 0.061, 0.05, 0.177, 0.054, 0.043...","[0.0, 7.70706, 20.81393, 50.82939, 80.35748, 9..."
3285,3286,Neeme Jarvi,Hary Janos Suite : VI. A csaszari udvar bevonu...,194.48118,"[0.0, 0.31692, 0.81846, 1.21107, 1.30313, 1.81...","[[1.0, 0.281, 0.083, 0.136, 0.167, 0.027, 0.02...","[0.0, 4.34719, 34.89175, 47.40584, 60.77849, 8..."
6214,6215,Dimitri Tiomkin,Villagers Galloping To Battle,192.75710,"[0.0, 0.37424, 0.75764, 0.93129, 1.23596, 1.36...","[[0.662, 0.712, 0.819, 1.0, 0.796, 0.781, 0.75...","[0.0, 7.87204, 15.60621, 25.77421, 38.15571, 6..."
9437,9438,Robert Lockwood_ Jr.,Kindhearted Woman Blues,162.45506,"[0.0, 0.37442, 0.80222, 1.06336, 1.69655, 2.10...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[0.0, 14.55654, 39.24917, 69.54568, 87.15621, ..."
5974,5975,All Time Low,Shameless,221.67465,"[0.0, 1.46535, 2.46395, 2.88712, 3.12522, 4.07...","[[0.071, 0.068, 1.0, 0.088, 0.099, 0.046, 0.13...","[0.0, 11.61403, 50.97236, 64.06348, 203.10287]"
6267,6268,Britt Nicole,Don't Worry Now (Performance Track In Key Of D...,245.81179,"[0.0, 0.199, 0.96059, 1.47696, 1.70376, 2.2267...","[[0.927, 0.8, 0.716, 0.605, 0.41, 0.549, 0.478...","[0.0, 17.48255, 36.64423, 55.95682, 97.2102, 1..."
6188,6189,4 Skins,Bread Or Blood (Single),170.52689,"[0.0, 0.45365, 0.55429, 0.87946, 3.33478, 3.96...","[[0.638, 0.379, 0.73, 0.378, 0.809, 1.0, 0.588...","[0.0, 7.75482, 14.75975, 45.25484, 54.46505, 9..."
824,825,Andy Russell,Easter Parade (1992 Digital Remaster),188.68200,"[0.0, 0.39179, 0.60721, 0.78063, 1.56032, 1.97...","[[0.828, 0.717, 0.824, 0.766, 0.938, 0.891, 1....","[0.0, 120.8006, 173.11189]"
4429,4430,Astarte,Risen From Within,173.81832,"[0.0, 0.24091, 0.60082, 0.96499, 1.34989, 1.69...","[[1.0, 1.0, 0.698, 0.819, 0.724, 0.907, 0.703,...","[0.0, 9.76361, 29.35627, 38.26336, 47.53061, 6..."
2530,2531,Eddie Cantor,Now's The Time To Fall In Love,155.61098,"[0.0, 0.15383, 0.65061, 0.94209, 1.36748, 1.65...","[[0.82, 1.0, 0.889, 0.895, 0.875, 0.925, 0.719...","[0.0, 7.64637, 28.35186, 53.52756, 87.17449, 9..."


In [168]:
import tensorflow as tf
import tensorflow.keras 
from spark_tensorflow_distributor import MirroredStrategyRunner
from keras.layers import Input, LSTM, Dense
import librosa
from sklearn.gaussian_process.kernels import WhiteKernel
import pyarrow as pa
import random
NOTECAP = 20
MAX_NOTE = newdf.iloc[-1,5].shape[0]
print(MAX_NOTE)


def partition_song(song, time, partition_size):
    
    def swapPitches(sample): # [12 x partition_size]
        
        res = [sample]
        for note in range(1,2):
            temp = np.copy(sample)
            for pitch in range(partition_size):
                temp[pitch, :] = np.array([temp[pitch,(i - note) % 12] for i in range(12)])
            res.append(temp)

        return res
    

    l, r = 0, partition_size
    # res = np.zeros(((song.shape[0] - partition_size)*(partition_size*12 + 1), song.shape[0], 13 + partition_size * 12))
    res = np.zeros((song[:50].shape[0]-partition_size , song[:50].shape[0], 13+12))
    # y = np.zeros(((song.shape[0] - partition_size)*(partition_size*12 + 1),2))
    y = np.zeros((song[:50].shape[0]-partition_size, 2))
    i = 0
    # for j in range(100):
    #     r = random.randint(1,song.shape[0]-partition_size-1)
    #     l = r + partition_size
    #     points.append([r,l])
    while r < len(song[:50]):
    # for l,r in points:
        sample = song[l:r] 
        for sample in [sample]:
        # for sample in swapPitches(sample):
            y[i,:] = [time[l]/time[-1], time[r]/time[-1]]
            for j in range(song[:50].shape[0]):                    
                res[i,j,:13] = np.concatenate([song[j], [time[j]/time[-1]], ])
                # res[i,j,:] = np.concatenate([song[j], [time[j]/time[-1]], sample.flatten()])

            # res[i,:partition_size,13:] = sample
            i+=1
        l+=1
        r+=1
    return res, y
    #if sample was 5 pitches long
    # we need to find all samples in the song of 5 pitches long
    # if song is 240 pitches
    # start at pitch 0
    # ends at pitch 235
def get_train(df):
    inputs = np.zeros((1, 50, 13+12))
    outputs = np.zeros((1, 2))
    k = 0
    for i,r in df.iloc[::-1].iterrows():
        if(k % 100 == 0):
            print(k)
        if(k == 2001):
            break
        for c in [10]:
            if(r["segments_start"].shape[0] > c):
                ans, yans = partition_song(r["segments_pitches"], r["segments_start"], c)
                # ans = np.append(ans,np.zeros((ans.shape[0],MAX_NOTE-ans.shape[1],13+12)),axis=1)
                inputs = np.append(inputs,ans,axis=0)
                outputs = np.append(outputs,yans,axis=0)
        if(k % 2000 == 0):
            np.save(open(f"data/inputs/song{k}.npy", "wb+"), inputs)
            np.save(open(f"data/outputs/song{k}.npy", "wb+"), outputs)
            inputs = np.zeros((1, 50, 13+12))
            outputs = np.zeros((1, 2))
        k += 1
    return np.array(inputs), np.array(outputs)

def edit_distance(sample, currMelody):

    return librosa.core.dtw(np.array(sample), np.array(currMelody))

    # if sample was 6 pitches long
# [[sample, timestamp of current song, pitch of current song][sample, timestamp + 1, pitch+1]],
#[[sample+1, timestamp of current song, pitch of current song][sample+1, timestamp + 1, pitch+1]]
# sample is list of pitches[]
# x = [] is for the number of segments per series
# y = [[0, 1]]
# when they give you a sample, it has a fixed number of chords/pitches
# [[[pitches1, pitches2, pitches3, pitches4, pitches5], [pitches2, pitches3, pitches4, pitches5, pitches6]]]

683


In [169]:
train_data = get_train(newdf)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [170]:

import numpy as np
x = np.load("data/inputs/song50.npy")
y = np.load("data/outputs/song50.npy")

x = x[1:]
y = y[1:]

# m = []
# step = 520
# for i in range(0,len(x),step):
#     j = 0
#     mx = 0
#     while j < x.shape[1] and np.sum(x[i, j, :]) > 0 :
#         j += 1
#         mx = max(mx, x[i,j,12])
#     m.append(mx)
# #m every 200, m[0] y[0:200], m
# y = np.array([[y[i,0]/m[i//step], y[i,1]/m[i//step]] for i in range(len(y))])
# for i in range(len(m)):
#     x[i*step: (i+1)*(step), :, 12] = x[i*step:(i+1)*(step), :,12] / m[i] 
# print(m)
# print(x.shape)
# print(y.shape)

In [171]:
%load_ext tensorboard
import tensorflow as tf
from keras.models import Model
import datetime
from tensorflow.keras.layers import Dense, Flatten,Reshape, Input, LSTM, GRU, Attention
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

input1 = Input(shape=(None,25))
input2 = Input(shape=(None,120))
# a = LSTM(10, activation="relu", return_sequences=True)(input)
a = LSTM(16, activation="relu")(input1)
b = Dense(16, activation="relu")(input2)
att = Attention(score_mode="concat")([a,b])
output1 = Dense(1, activation='linear')(att)
output2 = Dense(1, activation='linear')(att)
#output1 = Dense(1, activation='linear')(a)
#output2 = Dense(1, activation='linear')(a)
# reshape1 = Reshape((1,2))(dense1)
# model = Model([input1,input2],[output1,output2])
model = Model([input1,input2],[output1,output2])
model.summary()

op = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=op,  loss="mae" ,metrics=[keras.metrics.MeanAbsolutePercentageError()])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Model: "model_42"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_117 (InputLayer)      [(None, None, 25)]           0         []                            
                                                                                                  
 input_118 (InputLayer)      [(None, None, 120)]          0         []                            
                                                                                                  
 lstm_61 (LSTM)              (None, 16)                   2688      ['input_117[0][0]']           
                                                                                                  
 dense_121 (Dense)           (None, None, 16)             1936      ['input_118[0][0]']           
 

In [173]:
import keras
callback = keras.callbacks.EarlyStopping(monitor='loss',
                                               patience=3)
model.fit([x[:,:,:],x[:,:10,13:].reshape(x.shape[0], 120)],(y[:,0], y[:,1]),batch_size=32,epochs=30,validation_split=.2, callbacks=[callback])
model.save("model.keras")

Epoch 1/30
50/50 [==============================] - 1s 11ms/step - loss: 0.0318 - dense_122_loss: 0.0157 - dense_123_loss: 0.0161 - dense_122_mean_absolute_percentage_error: 725331.0625 - dense_123_mean_absolute_percentage_error: 48.2465 - val_loss: 0.0333 - val_dense_122_loss: 0.0165 - val_dense_123_loss: 0.0168 - val_dense_122_mean_absolute_percentage_error: 746762.8750 - val_dense_123_mean_absolute_percentage_error: 49.4734
Epoch 2/30
50/50 [==============================] - 0s 10ms/step - loss: 0.0318 - dense_122_loss: 0.0157 - dense_123_loss: 0.0161 - dense_122_mean_absolute_percentage_error: 720013.6250 - dense_123_mean_absolute_percentage_error: 47.7004 - val_loss: 0.0333 - val_dense_122_loss: 0.0165 - val_dense_123_loss: 0.0168 - val_dense_122_mean_absolute_percentage_error: 738111.3750 - val_dense_123_mean_absolute_percentage_error: 49.4902
Epoch 3/30
50/50 [==============================] - 1s 11ms/step - loss: 0.0318 - dense_122_loss: 0.0157 - dense_123_loss: 0.0161 - dense_

In [209]:
o = df.loc[df['artist'] == 'Rick Astley']
o = o.loc[5552,"segments_pitches"]
#6152
newX = np.zeros((1,50,25))
newX[:12] = np.concatenate([:50], )


model.predict()

,pk,artist,title,duration,segments_start,segments_pitches,section_start
851,852,Kanye West,Street Lights,189.96200,"[0.0, 0.19651, 0.36871, 0.60095, 0.68195, 0.84...","[[0.982, 1.0, 0.432, 0.384, 0.316, 0.309, 0.32...","[0.0, 8.12651, 28.26032, 76.0171, 97.51632, 16..."
1743,1744,Kanye West,School Spirit Skit 2,46.60200,"[0.0, 0.21937, 0.40381, 0.59465, 0.94317, 1.14...","[[0.165, 0.189, 0.365, 1.0, 0.704, 0.124, 0.05...","[0.0, 11.04137, 40.0806]"
4590,4591,Kanye West,Through The Wire,271.25506,"[0.0, 0.36313, 0.67442, 0.88785, 1.01011, 1.44...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[0.0, 7.8079, 37.21657, 62.62696, 70.09999, 87..."
6901,6902,Kanye West,Skit #2 (Kanye West/Late Registration),31.32036,"[0.0, 0.19442, 0.62544, 0.81737, 1.26145, 1.61...","[[1.0, 0.641, 0.114, 0.045, 0.039, 0.054, 0.05...","[0.0, 12.82242]"
7712,7713,Kanye West,Diamonds From Sierra Leone,247.64036,"[0.0, 1.36698, 1.7849, 2.12789, 2.45274, 2.835...","[[0.09, 0.065, 0.079, 0.087, 0.064, 0.089, 0.1...","[0.0, 12.07955, 39.69848, 60.52053, 122.28295,..."
9415,9416,Kanye West,Skit #2 (Kanye West/Late Registration),31.37261,"[0.0, 0.24658, 0.6258, 0.89084, 1.31215, 1.669...","[[1.0, 0.736, 0.116, 0.034, 0.027, 0.037, 0.03...","[0.0, 15.06622]"


In [125]:
model.predict()

[[1.         0.937      0.229      0.188      0.164      0.166
  0.167      0.142      0.137      0.116      0.094      0.286
  0.        ]
 [0.041      0.232      0.481      0.022      0.056      0.028
  1.         0.083      0.031      0.16       0.13       0.033
  0.0018502 ]
 [0.506      0.86       0.231      0.095      1.         0.192
  0.454      0.074      0.371      0.086      0.182      0.498
  0.00345886]
 [0.125      0.172      1.         0.14       0.242      0.058
  0.498      0.07       0.085      0.46       0.038      0.581
  0.00496079]
 [0.219      0.195      1.         0.226      0.061      0.053
  0.104      0.019      0.026      0.333      0.062      0.036
  0.00631552]
 [0.266      0.168      1.         0.277      0.05       0.029
  0.087      0.032      0.032      0.171      0.065      0.072
  0.00796582]
 [0.088      0.059      0.151      0.028      0.016      0.018
  0.125      1.         0.203      0.042      0.04       0.244
  0.00949905]
 [0.234      0.115  

array([[1.   , 0.937, 0.229, 0.188, 0.164, 0.166, 0.167, 0.142, 0.137,
        0.116, 0.094, 0.286],
       [0.041, 0.232, 0.481, 0.022, 0.056, 0.028, 1.   , 0.083, 0.031,
        0.16 , 0.13 , 0.033],
       [0.506, 0.86 , 0.231, 0.095, 1.   , 0.192, 0.454, 0.074, 0.371,
        0.086, 0.182, 0.498],
       [0.125, 0.172, 1.   , 0.14 , 0.242, 0.058, 0.498, 0.07 , 0.085,
        0.46 , 0.038, 0.581],
       [0.219, 0.195, 1.   , 0.226, 0.061, 0.053, 0.104, 0.019, 0.026,
        0.333, 0.062, 0.036],
       [0.266, 0.168, 1.   , 0.277, 0.05 , 0.029, 0.087, 0.032, 0.032,
        0.171, 0.065, 0.072],
       [0.088, 0.059, 0.151, 0.028, 0.016, 0.018, 0.125, 1.   , 0.203,
        0.042, 0.04 , 0.244],
       [0.234, 0.115, 0.303, 0.06 , 0.027, 0.024, 0.191, 1.   , 0.402,
        0.064, 0.097, 0.508],
       [0.431, 0.541, 0.49 , 0.134, 0.837, 0.326, 0.128, 0.331, 0.142,
        0.159, 0.157, 1.   ],
       [0.077, 0.07 , 0.158, 0.245, 1.   , 0.293, 0.018, 0.116, 0.041,
        0.051, 0.03 